In [3]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [4]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [5]:
df = pd.read_excel('ESTADIC//Base_ESTADIC_2019.xlsx', sheet_name='Direitos humanos', nrows=29, usecols=['CodUF', 'EDHU03','EDHU04','EDHU05', 'EDHU06', 'EDHU07'])
df

,CodUF,EDHU03,EDHU04,EDHU05,EDHU06,EDHU07
0,11,coordenadoria dos direitos humanos,Feminino,41,Parda,Ensino superior completo
1,12,sec. de estado assistência social dos dir. hum...,Feminino,40,Branca,Ensino superior completo
2,13,secretaria de justiça direitos humanos e cidad...,Feminino,34,Branca,Especialização
3,14,-,-,-,-,-
4,15,SECRETARIA DE JUSTIÇA E DIREITOS HUMANOS,Masculino,34,Parda,Especialização
5,16,secretaria estadual de justiça e segurança pub...,Masculino,53,Branca,Ensino superior completo
6,17,secretaria de estado da cidadania e justiça,Masculino,46,Branca,Mestrado
7,21,SEC DOS DIREITOS HUMANOS PARTICIPAÇAO POPULAR,Masculino,65,Parda,Doutorado
8,22,SECRETARIA DE ESTADO DA ASSISTÊNCIa social,Masculino,55,Parda,Especialização
9,23,"secretaria de proteção social, justiça, mulher...",Feminino,76,Branca,Mestrado


In [6]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2020.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])
uf

,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação
0,11,RO,Rondônia
1,12,AC,Acre
2,13,AM,Amazonas
3,14,RR,Roraima
4,15,PA,Pará
5,16,AP,Amapá
6,17,TO,Tocantins
7,21,MA,Maranhão
8,22,PI,Piauí
9,23,CE,Ceará


Renomeando as colunas

In [7]:
df.columns

Index(['CodUF', 'EDHU03', 'EDHU04', 'EDHU05', 'EDHU06', 'EDHU07'], dtype='object')

In [8]:
df= df.rename(columns={'CodUF':'cod_uf',
                       'EDHU03':'caracterizacao_orgao_gestor',
                        'EDHU04':'genero',
                        'EDHU05':'idade',
                        'EDHU06':'cor_raca',
                        'EDHU07':'grau_instrucao'}) 

In [9]:
df['ano']=2019

In [10]:
x= uf.pivot_table(columns=('Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação'), aggfunc='size')


In [11]:
uf = pd.DataFrame(x).reset_index()[['Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação']]

In [12]:
df = df.merge(uf, right_on='Código da Unidade da Federação',left_on='cod_uf') #juntando os dataframes, adicionando sigla e nome das UFs


In [13]:
df = df.drop(['Código da Unidade da Federação'], axis=1) #eliminando coluna repetida

In [14]:
df = df.rename(columns={'Sigla da Unidade da Federação':'sigla_uf',
                        'Nome da Unidade da Federação':'uf'}) #padronizando as colunas

In [15]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [16]:
#padronizando os dados da coluna 'tipo_orgao'
df['caracterizacao_orgao_gestor']=np.where(df['caracterizacao_orgao_gestor']=='-','Sem dados',df['caracterizacao_orgao_gestor'])


#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])

#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])

#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])


#padronizando os dados da coluna 'escolaridade'
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])

In [17]:
limites = [0, 30, 50,65,100] #criando uma nova coluna (faixa_etaria) com base na coluna 'idade'
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [18]:
df['grau_instrucao'].unique()

array(['Ensino superior completo', 'Especialização', 'Sem dados',
       'Mestrado', 'Doutorado', 'Ensino superior incompleto'],
      dtype=object)

In [19]:
dict_esco = {'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [20]:
df = df.replace({'grau_instrucao':dict_esco})

In [21]:
df= df[['ano', 'sigla_uf','cod_uf', 'uf', 'caracterizacao_orgao_gestor','genero', 'faixa_etaria', 'cor_raca','grau_instrucao']]

In [23]:
df.columns

Index(['ano', 'sigla_uf', 'cod_uf', 'uf', 'caracterizacao_orgao_gestor',
       'genero', 'faixa_etaria', 'cor_raca', 'grau_instrucao'],
      dtype='object')

In [38]:
df['cor_raca'].unique()

array(['Parda', 'Branca', 'Sem dados', 'Indígena', 'Preta'], dtype=object)

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          27 non-null     int64   
 1   sigla_uf                     27 non-null     object  
 2   cod_uf                       27 non-null     int64   
 3   uf                           27 non-null     object  
 4   caracterizacao_orgao_gestor  27 non-null     object  
 5   genero                       27 non-null     object  
 6   faixa_etaria                 26 non-null     category
 7   cor_raca                     27 non-null     object  
 8   grau_instrucao               27 non-null     object  
dtypes: category(1), int64(2), object(6)
memory usage: 2.1+ KB


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'), 
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'), 
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')
        ]


In [ ]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_direitos_humanos_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=90d2af81-d179-4a78-a596-3b9556bfd579>